In [2]:
from nnunet.evaluation.metrics import ConfusionMatrix, ALL_METRICS
import nibabel
import numpy as np
import os
from batchgenerators.utilities.file_and_folder_operations import save_json, subfiles, join
from tqdm import tqdm
import surface_distance as surfdist
import prettytable as pt
from skimage.morphology import skeletonize, skeletonize_3d

METRICS =[
    ['Accuracy','Precision','Recall'], # 体素精度，准确度(TP + TN) / (TP + FP + FN + TN)，精确度 TP / (TP + FP)，召回率(TP / (TP + FN))
    ['Dice','clDice'], # 相似度，骰子系数和中心线系数
    ['Hausdorff Distance 95','Avg. Symmetric Surface Distance'] # 距离精度,hd95,assd
] 

def cl_score(v, s):
    """[this function computes the skeleton volume overlap]
    Args:
        v ([bool]): [image]
        s ([bool]): [skeleton]
    Returns:
        [float]: [computed skeleton volume intersection]
    """
    return np.sum(v*s)/np.sum(s)


def clDice(v_p, v_l):
    """[this function computes the cldice metric]
    Args:
        v_p ([bool]): [predicted image]
        v_l ([bool]): [ground truth image]
    Returns:
        [float]: [cldice metric]
    """
    if len(v_p.shape)==2:
        tprec = cl_score(v_p,skeletonize(v_l))
        tsens = cl_score(v_l,skeletonize(v_p))
    elif len(v_p.shape)==3:
        tprec = cl_score(v_p,skeletonize_3d(v_l))
        tsens = cl_score(v_l,skeletonize_3d(v_p))
    return 2*tprec*tsens/(tprec+tsens)

def get_files_metrics(matrix,result=None):
    surface_distances = 0
    for item in result:
        if result[item]==None:
            result[item]=[]
                
        if item =='clDice':
            result[item].append(clDice(matrix['data'].test,matrix['data'].reference)*100)
            continue

        if item =='Hausdorff Distance 95':
            if surface_distances==0:
                surface_distances = surfdist.compute_surface_distances(matrix['data'].test==1, matrix['data'].reference==1, spacing_mm=matrix['voxel_spacing'])
            hd95 = surfdist.compute_robust_hausdorff(surface_distances, 95)
            result[item].append(hd95)
            continue

        if item =='Avg. Symmetric Surface Distance':
            if surface_distances==0:
                surface_distances = surfdist.compute_surface_distances(matrix['data'].test==1, matrix['data'].reference==1, spacing_mm=matrix['voxel_spacing'])                    
            asd = surfdist.compute_average_surface_distance(surface_distances)
            assd = (asd[0]+asd[1])/2
            result[item].append(assd)
            continue
        
        result[item].append(ALL_METRICS[item](confusion_matrix=matrix['data'])*100)
    return result

def get_fold_result(seg_fold,label_fold,cut):
    patients = os.listdir(seg_fold)
    result = {}.fromkeys([item for sublist in METRICS for item in sublist])
    matrix = []
    for i in tqdm(patients):
        try:
            seg = nibabel.load(join(seg_fold,i)).get_fdata().astype(np.uint8)
            label = nibabel.load(join(label_fold,i))
            voxel_spacing = label.header['pixdim'][1:4]
            label = label.get_fdata().astype(np.uint8)
            if cut:
                zz,_,_ = np.where(label)
                seg = seg[np.min(zz): np.max(zz)+1]
                label = label[np.min(zz): np.max(zz)+1]
            matrix={'data':ConfusionMatrix(test=seg==1,reference=label==1),'voxel_spacing':voxel_spacing}
            result = get_files_metrics(matrix,result)
        except:
            # print('missing '+i)
            continue
    return result

def evaluate(seg_dir,label_dir,fold_list=[0],cut=True):
    tb = pt.PrettyTable()
    tb.field_names = ['Fold Name']+[item for sublist in METRICS for item in sublist]
    results = {i:{} for i in fold_list}
    average_result_fold = []
    for fold in fold_list:
        result = get_fold_result(join(seg_dir,'fold'+str(fold)),label_dir,cut)
        result_list = []
        item = []
        for m in result:
            nums = result[m]
            result[m] = {'data':nums}
            nums = np.array(nums)
            result[m]['mean']=nums.mean()
            result[m]['std']=nums.std()
            result_list.append('{:.5f}±{:.5f}'.format(nums.mean(),nums.std()))
            item.append(nums.mean())
        tb.add_row(['fold'+str(fold)]+result_list)
        average_result_fold.append(item)
        results[fold]=result
        del result

    average_result_fold = np.stack(average_result_fold)
    avg_mean = average_result_fold.mean(axis=0)
    avg_std = average_result_fold.std(axis=0)
    average_result_fold = [ '{:.5f}±{:.5f}'.format(avg_mean[i],avg_std[i]) for i in range(len(tb.field_names)-1)]
    tb.add_row(['fold avg']+average_result_fold) 

    return results,tb

import pandas as pd

def tb_write_csv(file_name,prettyTable):
    data = prettyTable.get_csv_string().replace("'",'').split('\r\n')
    data = [ i.split(',') for i in data]
    df = pd.DataFrame(data = data[1:-1],columns=data[0])
    df.to_csv(file_name+'.csv')
    print('save successfully!')



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet



## Vnet

In [5]:
# seg_dir = r'/root/workspace/data/train_models/Task050/Vnet/'
# label_dir = r'/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr'
# fold_list = [0,1,2,3]
# result,tb = evaluate(seg_dir,label_dir,fold_list,True)
# print(tb)

task_name = 'Vnet'
seg_dir = '/root/workspace/data/train_models/Task050/'+task_name
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 12/12 [01:48<00:00,  9.04s/it]

+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |    Precision     |      Recall      |       Dice       |      clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
|   fold0   | 99.93274±0.02011 | 81.92032±5.91007 | 89.58818±5.20308 | 85.30634±3.06182 | 80.60350±5.37983 |   31.51061±18.13462   |         2.46200±1.17453         |
|   fold1   | 99.93996±0.02483 | 82.15997±6.71644 | 90.57166±4.24231 | 85.97256±4.28722 | 79.80988±6.31804 |   25.02923±11.15151   |         2.05579±0.69323         |
|   fold2   | 99.93735±0.02754 | 84.10374±9.68942 | 84.70249±5.72960 | 83.91515±5.65371 | 77.60292±8.74293 |   23.93268±10.46964   |         2.20678±0.79836         

## Vnet DCRE

In [27]:
task_name = 'vnet_dcre'
seg_dir = '/root/workspace/data/train_models/Task050/'+task_name
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 12/12 [01:38<00:00,  8.24s/it]

+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |    Precision     |      Recall      |       Dice       |      clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
|   fold0   | 99.94291±0.01596 | 85.93318±4.81498 | 88.82467±5.46995 | 87.12079±2.63462 | 85.76224±2.75972 |   12.25242±11.73230   |         1.28288±0.62752         |
|   fold1   | 99.94365±0.02333 | 82.96453±6.66665 | 91.43406±4.29787 | 86.79625±4.14332 | 82.98652±5.35917 |    14.48068±8.81843   |         1.32135±0.56852         |
|   fold2   | 99.94558±0.02180 | 87.66045±7.57336 | 83.93372±5.65774 | 85.51419±5.11397 | 83.28380±6.90577 |   15.70664±12.24221   |         1.43840±0.63770         

## MedNet

In [6]:
task_name = 'MedNet'
seg_dir = '/root/workspace/data/train_models/Task050/'+task_name
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 13/13 [01:26<00:00,  6.69s/it]

+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |    Precision     |      Recall      |       Dice       |      clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
|   fold0   | 99.93907±0.01421 | 89.15541±3.82369 | 82.13886±6.14888 | 85.29778±3.37189 | 81.66416±3.65840 |   22.39183±13.24043   |         1.61525±0.86042         |
|   fold1   | 99.94357±0.01695 | 88.26298±5.04429 | 83.46016±6.42021 | 85.54320±3.84173 | 80.17010±5.98232 |    13.99882±5.50248   |         1.18975±0.43822         |
|   fold2   | 99.94276±0.01255 | 91.39983±4.75910 | 77.66042±4.52339 | 83.79725±2.79280 | 79.53817±6.57237 |    17.98968±8.39081   |         1.53677±0.49541         

## DDT

In [7]:
task_name = 'ddt'
seg_dir = '/root/workspace/data/train_models/Task050/'+task_name
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 13/13 [01:21<00:00,  6.30s/it]

+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |    Precision     |      Recall      |       Dice       |      clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
|   fold0   | 99.94214±0.01891 | 91.30925±3.93650 | 81.92969±6.82058 | 86.06984±2.92684 | 83.08778±3.82634 |    11.29203±9.69580   |         1.11692±0.58383         |
|   fold1   | 99.94780±0.02251 | 90.03915±4.97721 | 84.43999±7.21307 | 86.85650±4.03478 | 82.80178±6.63534 |    6.13610±5.73514    |         0.82702±0.46865         |
|   fold2   | 99.94705±0.01580 | 92.09707±5.27683 | 79.29854±6.08043 | 85.00949±4.25732 | 80.54767±8.40021 |    11.16049±6.79895   |         1.44508±0.60266         

## nnUnet 

patch_size 64 128 128

epoch 100


In [5]:
task_name = 'baseline_smallsize'
seg_dir = '/root/workspace/data/INFER_DATA/Task050_pelvis/'+task_name
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 12/12 [01:12<00:00,  6.01s/it]

save successfully!


In [6]:
print(tb)

+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |    Precision     |      Recall      |       Dice       |      clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
|   fold0   | 99.95275±0.01903 | 91.18978±3.64452 | 87.61079±6.39653 | 89.11500±2.70940 | 88.23138±2.54854 |    5.90537±4.95436    |         0.79489±0.26279         |
|   fold1   | 99.95516±0.01732 | 88.35883±5.88480 | 90.08528±4.69554 | 89.01734±3.39199 | 86.74286±4.97654 |    4.79245±3.98740    |         0.78941±0.38947         |
|   fold2   | 99.95490±0.01577 | 91.39708±5.99401 | 84.94277±5.44728 | 87.80657±3.64906 | 84.81463±7.62806 |    8.93191±7.55228    |         1.11596±0.57030         

## Dilated Centerline

patch_size 64 128 128
epoch 100


In [7]:
task_name = 'DC_ss'
seg_dir = '/root/workspace/data/INFER_DATA/Task050_pelvis/'+task_name
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 12/12 [01:08<00:00,  5.71s/it]

+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |    Precision     |      Recall      |       Dice       |      clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
|   fold0   | 99.95604±0.01497 | 91.53188±3.64475 | 88.46061±5.88508 | 89.76048±2.55288 | 91.19293±1.64418 |    3.29957±3.08411    |         0.62745±0.31829         |
|   fold1   | 99.95494±0.01757 | 87.80805±6.02141 | 90.73343±4.67919 | 89.03709±3.36301 | 89.00304±5.24952 |    3.26335±3.36906    |         0.58282±0.32095         |
|   fold2   | 99.95657±0.01241 | 92.44345±4.51126 | 84.62668±5.18534 | 88.16344±2.76077 | 87.19635±5.53726 |    6.43843±5.38541    |         0.82651±0.37684         

## Dilated Centerline + Re

patch_size 64 128 128
epoch 100


In [8]:
task_name = 'DCRE_ss'
seg_dir = '/root/workspace/data/INFER_DATA/Task050_pelvis/'+task_name
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 12/12 [01:10<00:00,  5.85s/it]

+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |    Precision     |      Recall      |       Dice       |      clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
|   fold0   | 99.95409±0.01686 | 90.68908±3.72354 | 88.65992±6.12822 | 89.43342±2.64134 | 90.65393±2.06280 |    3.75589±3.00725    |         0.58854±0.21870         |
|   fold1   | 99.95704±0.01761 | 88.28222±5.94583 | 91.37728±4.13283 | 89.61306±3.16601 | 89.82812±4.91889 |    2.68865±2.50453    |         0.53959±0.29014         |
|   fold2   | 99.95717±0.01207 | 92.03411±4.25607 | 85.34170±5.13418 | 88.37491±2.71057 | 87.80473±5.42986 |    7.49924±7.24639    |         0.89641±0.64709         

### Fustion trainer

In [10]:
task_name = 'fusion_ss'
seg_dir = '/root/workspace/data/INFER_DATA/Task050_pelvis/'+task_name
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 12/12 [01:09<00:00,  5.77s/it]

+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |    Precision     |      Recall      |       Dice       |      clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
|   fold0   | 99.95517±0.01374 | 89.75611±3.62654 | 90.11689±5.26369 | 89.75923±2.27297 | 91.81683±1.60640 |    2.37464±1.73221    |         0.54418±0.16741         |
|   fold1   | 99.95554±0.01767 | 87.59257±6.25944 | 91.35022±4.03724 | 89.23319±3.32321 | 89.02256±5.42966 |    2.89359±2.51394    |         0.58648±0.30925         |
|   fold2   | 99.95762±0.01240 | 92.71089±4.24281 | 84.84047±5.28170 | 88.41650±2.88468 | 87.88923±5.30940 |    6.00143±5.78497    |         0.80735±0.41591         

### Fustion trainer + MSE

In [4]:
task_name = 'Fusion_ss_MSE'
seg_dir = '/root/workspace/data/INFER_DATA/Task050_pelvis/'+task_name
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [0,2]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 13/13 [01:03<00:00,  4.87s/it]

+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |    Precision     |      Recall      |       Dice       |      clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
|   fold0   | 99.95522±0.01754 | 91.33424±3.45033 | 88.62135±6.20260 | 89.72518±2.53097 | 91.52316±1.27202 |    2.41388±1.55645    |         0.51649±0.11503         |
|   fold2   | 99.95870±0.01179 | 91.77409±4.45873 | 86.51639±5.23900 | 88.86369±2.65468 | 89.09865±4.51964 |    4.78607±4.25227    |         0.64048±0.33826         |
|  fold avg | 99.95696±0.00174 | 91.55417±0.21992 | 87.56887±1.05248 | 89.29444±0.43074 | 90.31090±1.21225 |    3.59998±1.18609    |         0.57848±0.06200         

### Fustion trainer + DAW

In [12]:
task_name = 'Fusion_ss_DAW'
seg_dir = '/root/workspace/data/INFER_DATA/Task050_pelvis/'+task_name
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [0,2]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 13/13 [01:14<00:00,  5.73s/it]

+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |    Precision     |      Recall      |       Dice       |      clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
|   fold0   | 99.95709±0.01156 | 91.30865±3.57885 | 89.08589±4.95739 | 90.01478±1.98583 | 90.95050±1.37277 |    4.25268±5.62055    |         0.59752±0.26847         |
|   fold2   | 99.95834±0.01307 | 91.48893±4.92533 | 86.64791±5.14253 | 88.79850±2.98834 | 87.66855±5.64483 |    5.17852±4.26950    |         0.75394±0.41875         |
|  fold avg | 99.95771±0.00062 | 91.39879±0.09014 | 87.86690±1.21899 | 89.40664±0.60814 | 89.30953±1.64098 |    4.71560±0.46292    |         0.67573±0.07821         

## SK-Net

In [13]:
task_name = 'sk_trainer'
seg_dir = '/root/workspace/data/INFER_DATA/Task050_pelvis/'+task_name
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 12/12 [01:12<00:00,  6.04s/it]

+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |    Precision     |      Recall      |       Dice       |      clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
|   fold0   | 99.94436±0.03041 | 91.10319±3.46864 | 83.97723±8.77084 | 87.02172±4.87149 | 84.59743±3.51786 |    10.19110±6.48392   |         1.23864±0.51704         |
|   fold1   | 99.95149±0.01771 | 88.10861±5.94073 | 88.39766±5.48744 | 88.01001±3.49292 | 84.60119±4.86831 |    8.19625±4.59317    |         0.99250±0.40124         |
|   fold2   | 99.94499±0.02109 | 88.13440±7.71886 | 83.51526±6.01317 | 85.37476±4.51091 | 81.28888±7.77319 |   26.96541±21.23548   |         2.07741±0.83191         

## fusion_ss_no_re

关闭了ccp 去掉了tcl损失，只进行特征融合，融合模块只在最后一层开启，

由于之前模块中看到开启re之后assd有所下降

所以这里就关闭了re

【结果分析】

没想到从指标上来开，关闭ccp，模型效果会更好，

不同于dcre和dc，fusionNet中，打开ccp反而出现了全面抑制性能的情况

但是之前tcl实验中可以看到如果不打开ccp，模型结果会变得很差，模型根本无法学习到很好的效果，所以ccp在一定程度上是有效果的 但是这里为什么不生效了 很难讲

所以后续会额外开一个多层开启ccp和不开启ccp的实验，可能在复杂模型下，ccp是 有效果的

In [14]:
task_name = 'fusion_ss_no_re'
seg_dir = '/root/workspace/data/INFER_DATA/Task050_pelvis/'+task_name
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 12/12 [01:11<00:00,  5.99s/it]

+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |    Precision     |      Recall      |       Dice       |      clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
|   fold0   | 99.95476±0.01495 | 90.70384±3.89572 | 88.91522±5.83249 | 89.57527±2.35880 | 90.94485±1.43156 |    3.27679±2.39522    |         0.64396±0.27729         |
|   fold1   | 99.95648±0.01777 | 88.12753±6.14421 | 91.31952±4.17854 | 89.48668±3.16009 | 90.12322±4.95814 |    2.48601±2.05365    |         0.49557±0.24737         |
|   fold2   | 99.95759±0.01286 | 92.01219±4.91994 | 85.71054±5.13698 | 88.53662±2.87582 | 88.39286±5.15503 |    5.52191±5.32221    |         0.74759±0.36560         

## Fusion_ss_no_re_l2

关闭了ccp 去掉了tcl损失，只进行特征融合，融合模块在最后两层开启，

此实验持续验证ccp效果

【结果分析】

只能说略好一点,相比于比fusion_ss_no_re_l1来说，有一定提升

In [3]:
task_name = 'Fusion_ss_no_re_l2'
seg_dir = '/root/workspace/data/INFER_DATA/Task050_pelvis/'+task_name
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 12/12 [01:00<00:00,  5.06s/it]

+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |    Precision     |      Recall      |       Dice       |      clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
|   fold0   | 99.95519±0.01622 | 90.73181±3.78899 | 89.09297±6.11969 | 89.67517±2.61468 | 91.81310±1.55411 |    2.74363±1.95754    |         0.59593±0.24886         |
|   fold1   | 99.95533±0.01732 | 88.17993±6.02345 | 90.57325±4.28719 | 89.15476±3.10901 | 88.86526±5.21985 |    3.37831±3.01736    |         0.58824±0.29696         |
|   fold2   | 99.95877±0.01251 | 92.08837±4.78455 | 86.13315±5.15199 | 88.81661±2.97822 | 87.95661±5.48738 |    5.38448±5.01009    |         0.74165±0.35993         

## Branch l2

In [29]:
task_name = 'Branch_ss_l2'
seg_dir = '/root/workspace/data/INFER_DATA/Task050_pelvis/'+task_name
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 12/12 [01:10<00:00,  5.88s/it]

+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |    Precision     |      Recall      |       Dice       |      clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
|   fold0   | 99.95641±0.01340 | 91.41086±3.55973 | 88.70264±5.58833 | 89.84296±2.36807 | 91.34557±1.60583 |    2.92823±2.27816    |         0.54969±0.19591         |
|   fold1   | 99.95554±0.01784 | 87.89787±6.14341 | 91.07600±4.27204 | 89.25191±3.24422 | 89.47435±4.92896 |    3.34275±3.10894    |         0.59088±0.30868         |
|   fold2   | 99.95778±0.01349 | 91.68494±5.47958 | 86.24093±5.05302 | 88.64869±3.01635 | 88.02845±5.15595 |    4.85592±4.42624    |         0.69224±0.33220         

## Fusion_ss_no_re_l3


In [16]:
task_name = 'Fusion_ss_no_re_l3'
seg_dir = '/root/workspace/data/INFER_DATA/Task050_pelvis/'+task_name
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 12/12 [01:16<00:00,  6.42s/it]

+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |    Precision     |      Recall      |       Dice       |      clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
|   fold0   | 99.95681±0.01242 | 91.03404±3.59505 | 89.34981±5.11949 | 90.00815±2.07093 | 91.28862±1.48662 |    2.87077±2.25180    |         0.63023±0.21502         |
|   fold1   | 99.95653±0.01653 | 88.28766±5.80164 | 91.06060±4.43739 | 89.45229±3.06162 | 90.05529±4.46511 |    2.63880±2.12912    |         0.52021±0.25855         |
|   fold2   | 99.95917±0.01235 | 91.63775±4.84682 | 86.98796±5.14423 | 89.04079±2.78739 | 88.60356±5.18936 |    4.66241±4.43214    |         0.65670±0.32525         

## Branch l3

In [36]:
task_name = 'Branch_ss_l3'
seg_dir = '/root/workspace/data/INFER_DATA/Task050_pelvis/'+task_name
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 12/12 [01:11<00:00,  6.00s/it]

+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |    Precision     |      Recall      |       Dice       |      clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
|   fold0   | 99.95618±0.01255 | 90.50393±3.57634 | 89.59673±5.17755 | 89.87617±2.21676 | 91.80118±1.23732 |    2.44222±1.94992    |         0.50721±0.15088         |
|   fold1   | 99.95640±0.01753 | 88.31798±6.15354 | 90.85942±4.55477 | 89.36057±3.40194 | 89.60592±4.65823 |    2.95623±2.95493    |         0.54224±0.28368         |
|   fold2   | 99.95848±0.01257 | 92.21629±4.56390 | 85.88165±5.12722 | 88.74846±2.90551 | 87.81369±5.49260 |    5.19998±4.38971    |         0.73849±0.36424         

## Fusion_ss_no_re_l4

In [19]:
task_name = 'Fusion_ss_no_re_l4'
seg_dir = '/root/workspace/data/INFER_DATA/Task050_pelvis/'+task_name
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 12/12 [01:15<00:00,  6.28s/it]

+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |    Precision     |      Recall      |       Dice       |      clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
|   fold0   | 99.95568±0.01274 | 90.29493±3.94239 | 89.67020±5.34878 | 89.78731±2.29834 | 90.97232±1.74923 |    3.65937±3.88335    |         0.68199±0.34665         |
|   fold1   | 99.95551±0.01767 | 87.35327±6.06039 | 91.73952±4.14151 | 89.29709±3.25453 | 89.45631±4.62410 |    3.29832±2.50429    |         0.60326±0.27542         |
|   fold2   | 99.95873±0.01243 | 91.78306±4.72236 | 86.53936±5.27296 | 88.87259±2.84473 | 87.71963±5.24574 |    5.91483±4.49619    |         0.84943±0.48267         

## Branch_l4

In [32]:
task_name = 'Branch_ss_l4'
seg_dir = '/root/workspace/data/INFER_DATA/Task050_pelvis/'+task_name
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 12/12 [01:15<00:00,  6.32s/it]

+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |    Precision     |      Recall      |       Dice       |      clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
|   fold0   | 99.95568±0.01280 | 90.51472±3.89739 | 89.35078±5.35514 | 89.73533±2.28508 | 91.46822±1.57913 |    2.92818±2.27220    |         0.56859±0.23768         |
|   fold1   | 99.95616±0.01806 | 87.74699±6.05063 | 91.62764±4.05118 | 89.45383±3.23127 | 89.88003±4.61115 |    2.70371±2.22626    |         0.52144±0.28311         |
|   fold2   | 99.95808±0.01336 | 91.92779±5.06256 | 86.09860±5.22567 | 88.69910±3.01478 | 87.41461±5.57588 |    4.59634±3.68663    |         0.75869±0.33085         

## Fusion_ss_no_re_l5

In [17]:
task_name = 'Fusion_ss_no_re_l5'
seg_dir = '/root/workspace/data/INFER_DATA/Task050_pelvis/'+task_name
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [0]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 13/13 [01:36<00:00,  7.39s/it]

+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |    Precision     |      Recall      |       Dice       |      clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
|   fold0   | 99.95471±0.01476 | 90.50702±3.85893 | 89.00470±5.88342 | 89.53177±2.51325 | 90.58884±1.79465 |    3.66364±3.06304    |         0.69142±0.21611         |
|  fold avg | 99.95471±0.00000 | 90.50702±0.00000 | 89.00470±0.00000 | 89.53177±0.00000 | 90.58884±0.00000 |    3.66364±0.00000    |         0.69142±0.00000         |
+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------

## Branch_l5

In [30]:
task_name = 'Branch_ss_l5'
seg_dir = '/root/workspace/data/INFER_DATA/Task050_pelvis/'+task_name
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 12/12 [01:13<00:00,  6.10s/it]

+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |    Precision     |      Recall      |       Dice       |      clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
|   fold0   | 99.95763±0.01025 | 90.80083±3.64177 | 89.78129±4.57966 | 90.14221±2.04303 | 91.25893±1.39999 |    2.89713±2.50379    |         0.54277±0.20652         |
|   fold1   | 99.95688±0.01763 | 88.65919±5.94492 | 90.78908±4.26111 | 89.52051±3.22103 | 89.19023±5.05692 |    2.68571±2.04483    |         0.54987±0.27600         |
|   fold2   | 99.95808±0.01339 | 91.07775±5.57424 | 87.12126±4.86695 | 88.82328±2.93885 | 87.52409±5.42708 |    4.92808±4.03289    |         0.76915±0.35696         

## FuBase_ss_l3_1

In [41]:
task_name = 'FuBase_ss_l3_1'
seg_dir = '/root/workspace/data/INFER_DATA/Task050_pelvis/'+task_name
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 12/12 [01:12<00:00,  6.02s/it]

+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |    Precision     |      Recall      |       Dice       |      clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
|   fold0   | 99.95592±0.01466 | 90.85308±3.55471 | 89.27294±5.60475 | 89.85379±2.23276 | 91.35987±1.14952 |    2.37060±1.46864    |         0.55018±0.13143         |
|   fold1   | 99.95744±0.01700 | 88.09535±5.97991 | 91.75978±4.11844 | 89.69949±3.18612 | 90.30799±4.78338 |    2.58597±2.46854    |         0.53386±0.27711         |
|   fold2   | 99.95889±0.01304 | 92.14467±4.96474 | 86.28406±5.12491 | 88.90828±2.94585 | 87.98386±5.13248 |    5.75077±6.61394    |         0.76114±0.39174         

## FuBase_ss_l3_2

In [2]:
task_name = 'FuBase_ss_l3_2'
seg_dir = '/root/workspace/data/INFER_DATA/Task050_pelvis/'+task_name
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 12/12 [01:15<00:00,  6.28s/it]

+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |    Precision     |      Recall      |       Dice       |      clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
|   fold0   | 99.95774±0.01231 | 91.54440±3.45980 | 89.08762±5.53082 | 90.11963±2.46762 | 91.85897±1.28580 |    3.36512±3.21589    |         0.56630±0.20226         |
|   fold1   | 99.95553±0.01709 | 88.03438±6.02116 | 90.81662±4.29269 | 89.20023±3.15079 | 89.25793±5.23430 |    2.79511±2.32223    |         0.55949±0.25638         |
|   fold2   | 99.95829±0.01256 | 91.72301±4.85000 | 86.39406±5.25766 | 88.76137±2.82929 | 88.12972±4.97502 |    5.58381±5.46205    |         0.77013±0.40893         

## FuBase_ss_l3_3

In [4]:
task_name = 'FuBase_ss_l3_3'
seg_dir = '/root/workspace/data/INFER_DATA/Task050_pelvis/'+task_name
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 12/12 [01:07<00:00,  5.60s/it]

+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |    Precision     |      Recall      |       Dice       |      clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
|   fold0   | 99.95727±0.01122 | 91.39175±3.54294 | 89.02750±5.09299 | 90.02577±2.15858 | 91.22971±1.68274 |    3.03561±2.38092    |         0.56895±0.17591         |
|   fold1   | 99.95672±0.01690 | 88.63608±5.91812 | 90.77643±3.95902 | 89.50313±2.96915 | 89.52985±4.81119 |    3.20756±2.73776    |         0.59436±0.34121         |
|   fold2   | 99.95691±0.01303 | 91.72160±4.66503 | 85.65043±5.42174 | 88.37185±2.92754 | 87.87342±5.39248 |    5.34899±4.64102    |         0.77410±0.38643         

## FuV3_ss_l3_1_02

In [5]:
task_name = 'FuV3_ss_l3_1_02'
seg_dir = '/root/workspace/data/INFER_DATA/Task050_pelvis/'+task_name
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [0,1]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 13/13 [01:25<00:00,  6.59s/it]

+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |    Precision     |      Recall      |       Dice       |      clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
|   fold0   | 99.95759±0.01311 | 91.48748±3.54226 | 89.19325±5.42272 | 90.14298±2.32419 | 92.02182±1.41778 |    2.41382±1.73005    |         0.46326±0.11676         |
|   fold1   | 99.95673±0.01680 | 88.35955±5.83600 | 91.15455±4.37066 | 89.52961±2.97576 | 89.80629±5.17325 |    2.99369±2.87662    |         0.53361±0.28822         |
|  fold avg | 99.95716±0.00043 | 89.92351±1.56396 | 90.17390±0.98065 | 89.83629±0.30669 | 90.91406±1.10776 |    2.70375±0.28994    |         0.49843±0.03518         

## FuV3_ss_l3_1_01

In [8]:
task_name = 'FuV3_ss_l3_1_01'
seg_dir = '/root/workspace/data/INFER_DATA/Task050_pelvis/'+task_name
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 12/12 [01:15<00:00,  6.29s/it]

+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |    Precision     |      Recall      |       Dice       |      clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
|   fold0   | 99.95781±0.01132 | 91.41711±3.48497 | 89.29947±5.04937 | 90.17821±2.07088 | 91.79066±1.28987 |    2.49477±1.70790    |         0.47864±0.11913         |
|   fold1   | 99.95667±0.01714 | 88.45872±5.96472 | 90.91057±4.26327 | 89.46803±3.13342 | 89.89562±5.01332 |    2.72820±2.42240    |         0.49386±0.27685         |
|   fold2   | 99.95839±0.01190 | 92.17824±4.48520 | 85.92648±5.06886 | 88.74627±2.62692 | 88.34224±5.02444 |    6.16257±6.52172    |         0.74723±0.42181         

In [9]:
task_name = 'FuV3_ss_l3_1_03'
seg_dir = '/root/workspace/data/INFER_DATA/Task050_pelvis/'+task_name
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 12/12 [01:09<00:00,  5.83s/it]

+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |    Precision     |      Recall      |       Dice       |      clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
|   fold0   | 99.95581±0.01315 | 90.60711±3.71537 | 89.35514±5.74955 | 89.76910±2.39853 | 91.58954±1.40818 |    3.39182±3.55552    |         0.55897±0.21856         |
|   fold1   | 99.95627±0.01711 | 88.21136±5.86638 | 90.99043±4.36361 | 89.38365±3.15655 | 89.43403±5.49016 |    3.01694±2.79570    |         0.54827±0.32369         |
|   fold2   | 99.95950±0.01187 | 92.33516±4.12713 | 86.26344±5.01963 | 89.02129±2.69757 | 87.99076±5.14897 |    5.49078±5.52584    |         0.78040±0.37531         

## Branch_ss_l1

In [42]:
task_name = 'Branch_ss_l1'
seg_dir = '/root/workspace/data/INFER_DATA/Task050_pelvis/'+task_name
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task050_pelvis/labelsTr/'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 12/12 [01:10<00:00,  5.90s/it]

+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |    Precision     |      Recall      |       Dice       |      clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+------------------+------------------+------------------+------------------+-----------------------+---------------------------------+
|   fold0   | 99.95712±0.01007 | 90.62763±3.76250 | 89.73555±4.76142 | 90.02103±2.11939 | 91.63734±1.11849 |    2.65657±2.22468    |         0.54706±0.19494         |
|   fold1   | 99.95381±0.01819 | 86.85088±6.29197 | 91.50817±4.16140 | 88.90900±3.30588 | 88.81679±4.75140 |    4.17942±4.06562    |         0.63818±0.36943         |
|   fold2   | 99.95872±0.01221 | 92.69928±3.94541 | 85.44484±5.32497 | 88.74328±2.80225 | 87.70224±4.97878 |    5.30999±5.45196    |         0.74256±0.36230         

# Portalvein

## DDT

In [3]:
task_name = 'ddt'
seg_dir = '/root/workspace/data/train_models/Task002/'+task_name
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task002_PortalVein/labelsTr'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 6/6 [00:34<00:00,  5.76s/it]

+-----------+------------------+-------------------+-------------------+-------------------+-------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |     Precision     |       Recall      |        Dice       |       clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+-------------------+-------------------+-------------------+-------------------+-----------------------+---------------------------------+
|   fold0   | 99.83904±0.04113 | 79.92931±10.18461 | 63.31632±10.54565 |  69.20346±3.98434 |  59.53358±8.89987 |    20.81691±2.15913   |         3.58000±0.56719         |
|   fold1   | 99.82042±0.07250 | 74.18183±14.34894 | 37.11136±15.83889 | 46.13728±15.24350 | 32.05689±13.99970 |   46.51844±14.62569   |         10.74504±4.26026        |
|   fold2   | 99.75534±0.08134 | 73.29832±18.69438 | 42.01023±14.18499 | 52.61976±15.08614 | 38.70078±17.61515 |   41.51902±18.34515   |        1

## Mednet

In [4]:
task_name = 'mednet'
seg_dir = '/root/workspace/data/train_models/Task002/'+task_name
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task002_PortalVein/labelsTr'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 6/6 [00:34<00:00,  5.73s/it]

+-----------+------------------+-------------------+-------------------+-------------------+-------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |     Precision     |       Recall      |        Dice       |       clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+-------------------+-------------------+-------------------+-------------------+-----------------------+---------------------------------+
|   fold0   | 99.82938±0.04086 | 77.54940±10.45624 |  61.77017±9.54672 |  67.43227±3.70596 |  59.38859±7.98126 |    19.80038±5.48077   |         3.32054±0.64317         |
|   fold1   | 99.81398±0.07894 | 68.39502±15.55730 | 47.35101±21.19932 | 50.91379±13.77720 | 45.18426±14.57874 |   49.15993±39.53900   |         8.06077±4.84373         |
|   fold2   | 99.77926±0.07156 | 75.12783±13.15174 | 56.01848±10.10676 |  62.74274±7.69923 | 49.66391±10.28961 |   34.85980±18.32974   |         

## Vnet

In [6]:
task_name = 'vnet'
seg_dir = '/root/workspace/data/train_models/Task002/'+task_name
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task002_PortalVein/labelsTr'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 6/6 [00:46<00:00,  7.69s/it]

+-----------+------------------+-------------------+-------------------+-------------------+-------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |     Precision     |       Recall      |        Dice       |       clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+-------------------+-------------------+-------------------+-------------------+-----------------------+---------------------------------+
|   fold0   | 99.76056±0.03776 |  57.86639±8.78371 | 73.72771±11.11411 |  63.48319±4.22060 |  55.96972±7.20797 |   40.51405±10.93578   |         5.72036±1.53816         |
|   fold1   | 99.69827±0.11002 | 34.30394±19.17338 | 47.83787±24.64446 | 36.96897±18.86222 | 35.77309±18.09739 |   86.51272±57.24142   |         12.49610±7.93573        |
|   fold2   | 99.69379±0.10952 | 58.00339±16.52048 | 62.05287±10.04933 |  57.72472±9.32513 | 49.85326±13.32322 |   35.72565±14.34575   |         

## nnUnet

In [5]:
task_name = 'nnUnet'
seg_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task002_PortalVein/infer'
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task002_PortalVein/labelsTr'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 6/6 [00:30<00:00,  5.00s/it]

+-----------+------------------+-------------------+-------------------+-------------------+-------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |     Precision     |       Recall      |        Dice       |       clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+-------------------+-------------------+-------------------+-------------------+-----------------------+---------------------------------+
|   fold0   | 99.84121±0.03328 | 80.54269±10.81842 | 61.22125±10.20786 |  68.43669±5.62146 |  66.03686±7.54262 |    25.54376±9.45033   |         3.56967±1.15100         |
|   fold1   | 99.83103±0.04881 | 70.65796±18.99383 | 57.37860±22.30424 | 57.07624±11.28648 | 56.78740±11.58896 |   57.92913±65.26436   |         6.53217±4.54025         |
|   fold2   | 99.81636±0.06319 |  82.68577±7.87960 |  59.38831±7.79213 |  68.52262±5.30610 |  67.36354±8.75368 |   24.81834±19.08239   |         

## DC_modal

In [7]:
task_name = 'DC_modal'
seg_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task002_PortalVein/infer2'
label_dir = '/root/workspace/work/nnUNetFrame/DATASET/nnUNet_raw/nnUNet_raw_data/Task002_PortalVein/labelsTr'
fold_list = [0,1,2,3]
result,tb = evaluate(seg_dir,label_dir,fold_list,True)
print(tb)
tb_write_csv(os.path.join(seg_dir,task_name),tb)

100%|██████████| 6/6 [00:30<00:00,  5.02s/it]

+-----------+------------------+-------------------+-------------------+-------------------+-------------------+-----------------------+---------------------------------+
| Fold Name |     Accuracy     |     Precision     |       Recall      |        Dice       |       clDice      | Hausdorff Distance 95 | Avg. Symmetric Surface Distance |
+-----------+------------------+-------------------+-------------------+-------------------+-------------------+-----------------------+---------------------------------+
|   fold0   | 99.84148±0.03581 | 79.82149±11.07151 | 62.91831±11.94457 |  68.91005±6.48711 |  67.13004±8.04597 |    23.07382±8.01537   |         3.24158±0.96311         |
|   fold1   | 99.84021±0.04928 | 73.94813±17.20588 | 56.60179±23.45339 | 57.76239±13.39568 | 56.89644±13.49086 |   57.96264±58.08611   |         6.13938±4.03597         |
|   fold2   | 99.81949±0.06388 |  83.33068±8.23266 |  59.83153±8.31777 |  69.05226±5.86465 |  68.31479±7.12220 |   23.21433±16.75002   |         